In [1]:
!pip install -U openai

My Side Project:

AI Demos: https://www.aidemos.com/

AIDemos.com is your go-to directory for video demos of the latest AI tools. AI Demos's goal is to educate and inform about the possibilities of AI.

Youtube : https://www.youtube.com/@aidemos.futuresmart

In [1]:
import pandas as pd
df = pd.read_csv("bank_support_train.csv")
df.head(5)

,Support Query,Top Category,Sub Category
0,Can you explain the monthly maintenance fee on...,Fees and Charges,Understanding Fees
1,"I was charged a fee for an ATM withdrawal, why?",Fees and Charges,Understanding Fees
2,How do I dispute a transaction fee I believe i...,Fees and Charges,Dispute Charges
3,Are there any fees for using online banking?,Fees and Charges,Understanding Fees
4,What are the charges for a wire transfer?,Fees and Charges,Understanding Fees


###Format Dataset

In [4]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Top Category": "' + row['Top Category'] + '", "Sub Category": "' + row['Sub Category'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['Support Query']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

dataset = pd.read_csv('bank_support_train.csv')
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']

[{'role': 'user',
  'content': 'Can you explain the monthly maintenance fee on my account?'},
 {'role': 'assistant',
  'content': '{"Top Category": "Fees and Charges", "Sub Category": "Understanding Fees"}'}]

In [5]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])

{'Top Category': 'Fees and Charges', 'Sub Category': 'Understanding Fees'}

Create Train and Val Set

In [6]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=dataset['Top Category'],
    random_state=42  # for reproducibility
)

In [7]:
type(train_data[0])

dict

Create JSONL file

In [8]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [10]:
from openai import OpenAI
client = OpenAI(api_key="sk-xda15eTImv62SYVb1eXw_2lOEvH7oRfCUFadCTgoEjT3BlbkFJHHPrtTsKyGtST8dcRBU3BZBSvcrW06fHTy9UP2wbMA")

Upload Training and Validation File

In [11]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-kfuzx6lD90qny5LxYE0BFvgX
Validation file id: file-jGtf8jh1UrDyjgke0Wqfhe6M


Create Finetuning Job

In [12]:
suffix_name = "yt_tutorial"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-roXBYUdLRXvcsKnwvW1DJ9Nt', created_at=1723140035, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AVOdG0yGREZO41elnVjY6u7y', result_files=[], seed=1950692149, status='validating_files', trained_tokens=None, training_file='file-kfuzx6lD90qny5LxYE0BFvgX', validation_file='file-jGtf8jh1UrDyjgke0Wqfhe6M', estimated_finish=None, integrations=[], user_provided_suffix='yt_tutorial')

###All Finetuning  Jobs

In [17]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-roXBYUdLRXvcsKnwvW1DJ9Nt', created_at=1723140035, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AVOdG0yGREZO41elnVjY6u7y', result_files=[], seed=1950692149, status='running', trained_tokens=None, training_file='file-kfuzx6lD90qny5LxYE0BFvgX', validation_file='file-jGtf8jh1UrDyjgke0Wqfhe6M', estimated_finish=1723140612, integrations=[], user_provided_suffix='yt_tutorial'), FineTuningJob(id='ftjob-hwTlky2VLWbknr8l0k6Ib268', created_at=1723139674, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Unexpected file format, expected either prompt/completion pairs or chat messages.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_ep

###Retrieve Specific Job

In [19]:
response = client.fine_tuning.jobs.retrieve("ftjob-roXBYUdLRXvcsKnwvW1DJ9Nt")
response


FineTuningJob(id='ftjob-roXBYUdLRXvcsKnwvW1DJ9Nt', created_at=1723140035, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:yt-tutorial:9u1saBZL', finished_at=1723140619, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-AVOdG0yGREZO41elnVjY6u7y', result_files=['file-j5mAixU7JyWn8fbfQABe8EAg'], seed=1950692149, status='succeeded', trained_tokens=9123, training_file='file-kfuzx6lD90qny5LxYE0BFvgX', validation_file='file-jGtf8jh1UrDyjgke0Wqfhe6M', estimated_finish=None, integrations=[], user_provided_suffix='yt_tutorial')

In [21]:
client.fine_tuning.jobs.cancel("ftjob-roXBYUdLRXvcsKnwvW1DJ9Nt")

BadRequestError: Error code: 400 - {'error': {'message': 'Job has already completed: ftjob-roXBYUdLRXvcsKnwvW1DJ9Nt', 'type': 'invalid_request_error', 'param': 'fine_tune_id', 'code': 'invalid_cancel'}}

In [20]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:yt-tutorial:9u1saBZL


###Test Finetuned Model

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):

    formatted_message = [
        {
            "role": "user",
            "content": row['Support Query']
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content

In [18]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")

In [19]:
test_df = pd.read_csv("test_queries.csv")
store_predictions(test_df, fine_tuned_model_id)

fine_tuned_model_id ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc
